# RAG 통합 최종 버전
## 다현님 + 나현님 + 사용자 작업 통합

이 노트북은 세 분의 작업을 통합한 최종 버전입니다:
- **다현님 작업**: 본문/부칙 분리, 학위/정원 문장 생성, 본문 청킹
- **나현님 작업**: UpstageDocumentParseLoader 활용 (참고)
- **사용자 작업**: 표 데이터 처리 및 벡터화

### 통합 전략
1. PDF 본문과 부칙을 분리하여 청킹
2. CSV 표 데이터를 문장으로 변환
3. 모든 문서를 FAISS 벡터 스토어에 통합
4. RAG 파이프라인 구축 및 평가


In [17]:
# 환경 설정
import os
import sys
from pathlib import Path
from typing import List, Optional

import pandas as pd
import re

# Upstage API Key 설정
os.environ.setdefault("UPSTAGE_API_KEY", "up_EoF0I0CzeHxuDYmf0we544GMPCFIT")
UPSTAGE_API_KEY = os.environ.get("UPSTAGE_API_KEY", "")

# 작업 디렉토리 설정
CURRENT_DIR = Path(__file__).parent if "__file__" in globals() else Path.cwd()
if Path.cwd() != CURRENT_DIR:
    os.chdir(CURRENT_DIR)

print(f"✅ 작업 디렉토리: {CURRENT_DIR}")
print(f"✅ Upstage API Key: {UPSTAGE_API_KEY[:20]}...")


✅ 작업 디렉토리: c:\Users\janen\Documents\25-2 강의\자연어처리\프로젝트\ewha
✅ Upstage API Key: up_EoF0I0CzeHxuDYmf0...


In [18]:
# 필요한 라이브러리 설치
%pip install langchain_upstage langchain_community langchain-openai openai pdfplumber pandas faiss-cpu


Note: you may need to restart the kernel to use updated packages.


In [19]:
# ==================================================================
# 1. PDF 본문/부칙 분리 및 청킹 (다현님 작업 기반)
# ==================================================================

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

PDF_PATH = CURRENT_DIR / "ewha.pdf"

if not PDF_PATH.exists():
    raise FileNotFoundError(f"{PDF_PATH} 파일을 찾을 수 없습니다.")

print("📄 PDF 파일 로딩 중...")
loader = PyPDFLoader(str(PDF_PATH))
docs = loader.load()
docs = [d for d in docs if int(d.metadata["page"]) < 38]  # 부칙 이전 페이지만
print(f"✅ 총 {len(docs)}페이지 로드 완료")

# 전체 텍스트 병합
full_text = "\n".join([d.page_content for d in docs])

# "부칙" 등장 지점 찾기
split_index = full_text.find("부칙")
print(f"📌 부칙 시작 인덱스: {split_index}")

# 본문 / 부칙 분리
if split_index != -1:
    main_text = full_text[:split_index]
    appendix_text = full_text[split_index:]
else:
    main_text = full_text
    appendix_text = ""

print(f"📄 본문 길이: {len(main_text)}")
print(f"📄 부칙 길이: {len(appendix_text)}")

# 본문 청킹 (다현님 설정 사용)
main_doc = [Document(page_content=main_text)]
text_splitter_main = RecursiveCharacterTextSplitter(
    chunk_size=1200,
    chunk_overlap=300,
    separators=["별표", "\n\n제", "\n제", "\n\n", "\n", ". ", " "],
)
main_chunks = text_splitter_main.split_documents(main_doc)
print(f"✅ 본문 청크 수: {len(main_chunks)}")

# 부칙 청킹
if appendix_text:
    appendix_doc = [Document(page_content=appendix_text)]
    text_splitter_appendix = RecursiveCharacterTextSplitter(
        chunk_size=1200,
        chunk_overlap=300,
        separators=["부칙", "별표", "\n\n제", "\n제", "\n\n", "\n", ". ", " "],
    )
    appendix_chunks = text_splitter_appendix.split_documents(appendix_doc)
    print(f"✅ 부칙 청크 수: {len(appendix_chunks)}")
else:
    appendix_chunks = []

print(f"\n📦 총 텍스트 청크: {len(main_chunks) + len(appendix_chunks)}개")


📄 PDF 파일 로딩 중...
✅ 총 38페이지 로드 완료
📌 부칙 시작 인덱스: 20919
📄 본문 길이: 20919
📄 부칙 길이: 20222
✅ 본문 청크 수: 23
✅ 부칙 청크 수: 23

📦 총 텍스트 청크: 46개


In [20]:
# ==================================================================
# 2. CSV 표 데이터 처리 및 문장 생성 (다현님 + 사용자 작업 통합)
# ==================================================================

def coalesce(*values: object) -> str:
    """첫 번째 유효한 값을 반환"""
    for value in values:
        if value is None:
            continue
        if isinstance(value, float) and pd.isna(value):
            continue
        text = str(value).strip()
        if text:
            return text
    return ""

# 다현님 작업: 학위 문장 생성 방식
def build_degree_sentences_from_csv(df: pd.DataFrame, year: str = "최신 개정") -> List[str]:
    """CSV에서 학위 문장 생성 (다현님 방식)"""
    sentences = []
    for idx, row in df.iterrows():
        college = coalesce(row.get("설치대학"), row.get("대학"))
        degree = coalesce(row.get("학위_종류"), row.get("학위종류"), row.get("학사"))
        major = coalesce(row.get("학과_전공"), row.get("학과 또는 전공"), row.get("전공"))
        
        if not college or not degree or not major:
            continue
            
        sentence = f"{college}의 {major} 전공은 {degree} 학위를 수여한다. ({year})"
        sentences.append(sentence)
    return sentences

# 사용자 작업: 표 데이터 처리 함수들
def build_quota_text(row: pd.Series) -> str:
    """입학정원 텍스트 생성 (사용자 방식)"""
    year = coalesce(row.get("학년도"), "2019학년도")
    college = coalesce(row.get("대학"))
    faculty = coalesce(row.get("학부"))
    major = coalesce(row.get("학과_전공"), row.get("학과 또는 전공"))
    quota = coalesce(row.get("정원"), row.get("입학정원_명")) or "미상"
    
    prefix_parts = [year]
    if college:
        prefix_parts.append(college)
    if faculty:
        prefix_parts.append(faculty)
    prefix = " ".join(prefix_parts)
    return f"{prefix} 소속 {major}의 입학 정원은 {quota}명입니다."

def build_degree_text(row: pd.Series) -> str:
    """학위 텍스트 생성 (사용자 방식)"""
    college = coalesce(row.get("설치대학"))
    major = coalesce(row.get("학과_전공"))
    degree = coalesce(row.get("학위_종류"))
    quota = coalesce(row.get("입학정원_명"))
    period = coalesce(row.get("설치_운영기간"))
    
    sentence = f"{college} 소속 {major} 전공자에게 수여하는 학위는 {degree}입니다."
    extras = []
    if quota:
        extras.append(f"입학 정원은 {quota}명")
    if period:
        extras.append(f"설치·운영 기간은 {period}")
    if extras:
        sentence += " " + ", ".join(extras) + "입니다."
    return sentence

def build_contract_text(row: pd.Series) -> str:
    """계약학과 텍스트 생성 (사용자 방식)"""
    college = coalesce(row.get("설치대학"))
    form = coalesce(row.get("설치형태"))
    major = coalesce(row.get("학과_전공"))
    degree = coalesce(row.get("학위_종류"))
    quota = coalesce(row.get("입학정원_명"))
    period = coalesce(row.get("설치_운영기간"))
    
    parts = [f"계약학과 설치·운영 정보: 설치대학={college}"]
    if form:
        parts.append(f"설치형태={form}")
    if major:
        parts.append(f"학과/전공={major}")
    if degree:
        parts.append(f"수여 학위={degree}")
    if quota:
        parts.append(f"입학 정원={quota}명")
    if period:
        parts.append(f"설치·운영 기간={period}")
    return ", ".join(parts) + "."

# CSV 파일 처리
csv_documents = []

# 1) degrees.csv 처리 (다현님 방식 + 사용자 방식 결합)
degrees_path = CURRENT_DIR / "degrees.csv"
if degrees_path.exists():
    df_degrees = pd.read_csv(degrees_path, encoding='utf-8-sig')
    # 다현님 방식: 학위 문장 생성
    degree_sentences = build_degree_sentences_from_csv(df_degrees, "최신 개정")
    for sentence in degree_sentences:
        csv_documents.append(Document(
            page_content=sentence,
            metadata={"source": "[별표 2] 학사학위의 종류", "page": 51, "type": "degree"}
        ))
    print(f"✅ degrees.csv 처리 완료: {len(degree_sentences)}개 문장")
else:
    print("⚠️ degrees.csv 파일이 없습니다.")

# 2) 2019_quota.csv 처리
quota_path = CURRENT_DIR / "2019_quota.csv"
if quota_path.exists():
    df_quota = pd.read_csv(quota_path, encoding='utf-8-sig')
    for _, row in df_quota.iterrows():
        text = build_quota_text(row)
        csv_documents.append(Document(
            page_content=text,
            metadata={"source": "[별표 1] 2019학년도 입학정원", "page": 39, "type": "quota"}
        ))
    print(f"✅ 2019_quota.csv 처리 완료: {len(df_quota)}개 문서")
else:
    print("⚠️ 2019_quota.csv 파일이 없습니다.")

# 3) contract_dept.csv 처리
contract_path = CURRENT_DIR / "contract_dept.csv"
if contract_path.exists():
    df_contract = pd.read_csv(contract_path, encoding='utf-8-sig')
    for _, row in df_contract.iterrows():
        text = build_contract_text(row)
        csv_documents.append(Document(
            page_content=text,
            metadata={"source": "[별표 3] 계약학과 설치·운영", "page": 53, "type": "contract"}
        ))
    print(f"✅ contract_dept.csv 처리 완료: {len(df_contract)}개 문서")
else:
    print("⚠️ contract_dept.csv 파일이 없습니다.")

print(f"\n📦 총 CSV 문서: {len(csv_documents)}개")


✅ degrees.csv 처리 완료: 2개 문장
✅ 2019_quota.csv 처리 완료: 79개 문서
✅ contract_dept.csv 처리 완료: 1개 문서

📦 총 CSV 문서: 82개


In [21]:
# ==================================================================
# 3. 모든 문서 통합 및 FAISS 벡터 스토어 생성
# ==================================================================

from langchain_upstage import UpstageEmbeddings
from langchain_community.vectorstores import FAISS

# 임베딩 모델 로드
embeddings = UpstageEmbeddings(
    api_key=UPSTAGE_API_KEY,
    model="solar-embedding-1-large"
)
print("✅ Upstage 임베딩 모델 로드 완료")

# 모든 문서 통합
all_documents = main_chunks + appendix_chunks + csv_documents

# 메타데이터 추가 (본문/부칙 청크)
for i, doc in enumerate(main_chunks):
    if not hasattr(doc, 'metadata') or not doc.metadata:
        doc.metadata = {}
    doc.metadata["source"] = "ewha.pdf 본문"
    doc.metadata["type"] = "main_text"

for i, doc in enumerate(appendix_chunks):
    if not hasattr(doc, 'metadata') or not doc.metadata:
        doc.metadata = {}
    doc.metadata["source"] = "ewha.pdf 부칙"
    doc.metadata["type"] = "appendix_text"

# 유효한 문서만 필터링
valid_documents = [doc for doc in all_documents if doc.page_content and doc.page_content.strip()]
print(f"🧹 필터링 후 유효 문서 수: {len(valid_documents)}")

# 문서 타입별 통계
doc_types = {}
for doc in valid_documents:
    doc_type = doc.metadata.get("type", "unknown")
    doc_types[doc_type] = doc_types.get(doc_type, 0) + 1

print("\n📊 문서 타입별 통계:")
for doc_type, count in doc_types.items():
    print(f"  - {doc_type}: {count}개")

# FAISS 벡터 스토어 생성
# Windows 한글 경로 문제 해결: 영어 경로만 있는 디렉토리 사용
import platform
import tempfile

# Windows에서 한글 경로 문제를 피하기 위해 임시 디렉토리나 영어 경로 사용
if platform.system() == 'Windows':
    # 방법 1: 환경 변수 TEMP 사용 (보통 C:\Users\USERNAME\AppData\Local\Temp)
    temp_dir = Path(tempfile.gettempdir())
    # 프로젝트별 고유한 디렉토리명 생성
    vector_db_base = temp_dir / "rag_ewha_vectorstore"
    vector_db_base.mkdir(parents=True, exist_ok=True)
    vector_db_path = str(vector_db_base.resolve())
    print(f"📂 Windows 한글 경로 문제 해결: 임시 디렉토리 사용")
    print(f"📂 벡터 DB 경로: {vector_db_path}")
else:
    # Windows가 아닌 경우 상대 경로 사용
    VECTOR_DB_DIR = CURRENT_DIR / "vectorstore"
    VECTOR_DB_DIR.mkdir(parents=True, exist_ok=True)
    vector_db_path = str(VECTOR_DB_DIR.resolve())

vector_store = None
vector_db_dir = Path(vector_db_path)
if vector_db_dir.exists() and (vector_db_dir / "index.faiss").exists():
    try:
        vector_store = FAISS.load_local(
            folder_path=vector_db_path,
            embeddings=embeddings,
            allow_dangerous_deserialization=True,
        )
        print(f"\n📂 기존 벡터 DB 로드 완료")
    except Exception as exc:
        print(f"⚠️ 기존 벡터 DB 로드 실패: {exc}. 새로 생성합니다.")
        vector_store = None

if vector_store is None:
    print(f"\n📦 총 {len(valid_documents)}개의 문서를 벡터화 중입니다...")
    vector_store = FAISS.from_documents(valid_documents, embeddings)
    # 영어 경로 사용 (한글 경로 문제 해결)
    vector_store.save_local(vector_db_path)
    print(f"✅ 벡터 DB 생성 완료")

print(f"\n🎯 총 KB 문서 수: {len(valid_documents)}개")


✅ Upstage 임베딩 모델 로드 완료
🧹 필터링 후 유효 문서 수: 128

📊 문서 타입별 통계:
  - main_text: 23개
  - appendix_text: 23개
  - degree: 2개
  - quota: 79개
  - contract: 1개
📂 Windows 한글 경로 문제 해결: 임시 디렉토리 사용
📂 벡터 DB 경로: C:\Users\janen\AppData\Local\Temp\rag_ewha_vectorstore

📦 총 128개의 문서를 벡터화 중입니다...
✅ 벡터 DB 생성 완료

🎯 총 KB 문서 수: 128개


In [22]:
# ==================================================================
# 4. RAG 파이프라인 구성 및 실행
# ==================================================================

from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage

# 테스트셋 경로
TESTSET_PATH = CURRENT_DIR.parent / "testset.csv"
TOP_K = 5  # 검색 문서 수 증가

# 리트리버 설정
retriever = vector_store.as_retriever(search_kwargs={"k": TOP_K})

# LLM 및 프롬프트 템플릿 설정
llm = ChatUpstage(api_key=UPSTAGE_API_KEY, model="solar-pro2")

prompt_template = PromptTemplate.from_template(
    """
    다음 문맥을 바탕으로 질문에 가장 정확하게 답변하세요.
    답변이 문맥에 없다면 정확히 "The information is not present in the context."라고 답변하세요.
    최종 답변만 제공하세요.
    
    ---
    Question: {question}
    ---
    Context:
    {context}
    """
)
chain = prompt_template | llm

print(f"✅ RAG 체인 구성 완료 (Model: solar-pro2, Top-k: {TOP_K})")

# 테스트셋 로드
def read_data(file_path: Path):
    data = pd.read_csv(file_path)
    return data["prompts"], data.get("answers")

try:
    prompts, answers = read_data(TESTSET_PATH)
    print(f"✅ 테스트 프롬프트 {len(prompts)}개 로드 완료")
except FileNotFoundError:
    print(f"❌ {TESTSET_PATH} 파일을 찾을 수 없습니다.")
    prompts, answers = [], None

# 테스트 실행 (처음 5개만)
if prompts is not None and len(prompts) > 0:
    print("\n▶ 샘플 테스트 시작...\n")
    
    for i, prompt in enumerate(prompts[:5], 1):
        source_docs = retriever.invoke(prompt)
        context_text = "\n\n".join(doc.page_content for doc in source_docs) or ""
        response = chain.invoke({"question": prompt, "context": context_text})
        
        print(f"Q{i}: {prompt[:60]}...")
        print(f"응답: {response.content[:150]}...")
        print(f"검색 문서: {len(source_docs)}개\n")
    
    print("✅ 샘플 테스트 완료")
else:
    print("⚠️ 실행할 프롬프트가 없습니다.")


✅ RAG 체인 구성 완료 (Model: solar-pro2, Top-k: 5)
✅ 테스트 프롬프트 50개 로드 완료

▶ 샘플 테스트 시작...

Q1: QUESTION1) 재학 중인 학생이 휴학을 하려면 학기 개시일로부터 며칠 이내에 휴학을 신청하야하나요?
(...
응답: (D) 90일  

**Answer:** (D) 90일  

**Context 근거:**  
제26조(휴학) ⑥ 재학 중인 자가 휴학을 하고자 하는 경우 학기개시일로부터 90일 이내에 휴학을 신청하여야 한다. (신설 2015.9.18.)  

이 조항은 재학 중인 학생...
검색 문서: 5개

Q2: QUESTION2) '재입학은 a회에 한하여 할 수 있다. 다만 제 28조제4호에 의하여 제적된 자는 제적된...
응답: 제31조제2항에 따르면, 재입학은 1회에 한하여 할 수 있다고 명시되어 있습니다. 또한 제28조제4호에 의하여 제적된 자는 제적된 날부터 1년이 경과한 후 재입학할 수 있다고 규정되어 있습니다. 

따라서, a는 1이고 b는 1이므로 a+b의 값은 2입니다.

정답: ...
검색 문서: 5개

Q3: QUESTION3) 학생이 소속 학과 또는 전공 이외의 전공 교과목을 총장이 정하는 바에 따라 몇학점 이상 ...
응답: (C) 21학점  

**Context 근거:**  
제47조(전공, 부전공 등) ③항에서 "학생이 소속 학과 또는 전공 이외의 전공 교과목을 총장이 정하는 바에 따라 **21학점 이상** 취득한 때에는 부전공을 이수한 것으로 인정한다"고 명시되어 있습니다. 따라서 정...
검색 문서: 5개

Q4: QUESTION4) 다음 보기의 학생들 중 제적을 당하지 않는 사람을 고르면?
(A) 팜 : 징계에 의해 퇴...
응답: (D) 로라 : 휴학기간 경과 후 3주가 지났으나 갑자기 큰 사고가 났다는 정당한 이유 때문에 복학하지 못하였음  
**The information is not present in the context.**  

**Explanation:*

In [23]:
# ==================================================================
# 5. 전체 테스트셋 평가 (다현님 + 나현님 방식 결합)
# ==================================================================

def extract_answer(response: str) -> Optional[str]:
    """
    응답에서 답변 추출 (다현님 + 나현님 방식 결합)
    - [ANSWER]: (A) 형식 우선
    - (A), (B), (C), (D) 패턴 찾기
    """
    # 우선순위 1: [ANSWER]: (X) 형식
    pattern1 = r"\[ANSWER\]:\s*\(([A-DE])\)"
    match = re.search(pattern1, response)
    if match:
        return match.group(1)
    
    # 우선순위 2: (X) 형식 (마지막 발생)
    pattern2 = r"\(([A-DE])\)"
    matches = list(re.finditer(pattern2, response))
    if matches:
        return matches[-1].group(1)
    
    # 우선순위 3: 단독 문자 (마지막 발생)
    pattern3 = r"\b([A-DE])\b(?!.*\b[A-DE]\b)"
    match = re.search(pattern3, response)
    if match:
        return match.group(1)
    
    return None

# 전체 평가 실행
if prompts is not None and len(prompts) > 0:
    print("====== RAG Evaluation (전체 테스트셋) ======\n")
    print(f"📋 총 {len(prompts)}개 문제 평가 시작...\n")
    
    all_responses = []
    predictions = []
    
    for i, prompt in enumerate(prompts, 1):
        source_docs = retriever.invoke(prompt)
        context_text = "\n\n".join(doc.page_content for doc in source_docs) or ""
        response = chain.invoke({"question": prompt, "context": context_text})
        
        pred_raw = response.content.strip()
        pred = extract_answer(pred_raw)
        
        predictions.append(pred)
        all_responses.append({
            "question_num": i,
            "question": prompt[:100],
            "prediction": pred,
            "response": pred_raw[:200],
            "context_count": len(source_docs)
        })
        
        # 진행 상황 출력 (모든 문제에 대해)
        gold = answers.iloc[i-1] if answers is not None else "N/A"
        status = "✅" if pred and str(pred) == str(gold) else "❌" if pred else "⚠️"
        print(f"Q{i}/{len(prompts)} {status} Pred: {pred}, Gold: {gold}")
        
        # 10개마다 상세 정보 출력
        if i % 10 == 0 or i == len(prompts):
            print(f"  └─ 진행률: {i}/{len(prompts)} ({i/len(prompts)*100:.1f}%)")
            print("-" * 40)
    
    # 정확도 계산 (전체 데이터셋)
    if answers is not None:
        correct = 0
        total = min(len(predictions), len(answers))
        no_answer = 0
        
        print(f"\n{'='*60}")
        print(f"📊 정확도 계산 (전체 {total}개 문제)")
        print(f"{'='*60}")
        
        for i in range(total):
            pred = str(predictions[i]).strip() if predictions[i] else None
            gold = str(answers.iloc[i]).strip()
            
            # (A) 형식에서 A만 추출
            gold_match = re.search(r"\(([A-DE])\)", gold)
            gold_letter = gold_match.group(1) if gold_match else gold
            
            if not pred:
                no_answer += 1
            elif pred and pred.upper() == gold_letter.upper():
                correct += 1
        
        accuracy = (correct / total * 100) if total > 0 else 0
        answer_rate = ((total - no_answer) / total * 100) if total > 0 else 0
        
        print(f"\n✅ 정답: {correct}개")
        print(f"❌ 오답: {total - correct - no_answer}개")
        print(f"⚠️  미응답: {no_answer}개")
        print(f"📝 총 문제: {total}개")
        print(f"\n🎯 전체 정확도: {accuracy:.2f}% ({correct}/{total})")
        print(f"📊 응답률: {answer_rate:.2f}% ({(total - no_answer)}/{total})")
    
    # 결과 저장
    results_df = pd.DataFrame(all_responses)
    results_path = CURRENT_DIR / "rag_results_combined.csv"
    results_df.to_csv(results_path, index=False, encoding='utf-8-sig')
    print(f"\n💾 결과 저장: {results_path}")
else:
    print("⚠️ 평가할 프롬프트가 없습니다.")


====== RAG Evaluation (전체 테스트셋) ======

📋 총 50개 문제 평가 시작...

Q1/50 ❌ Pred: D, Gold: (D)
Q2/50 ❌ Pred: A, Gold: (A)
Q3/50 ❌ Pred: C, Gold: (C)
Q4/50 ❌ Pred: D, Gold: (D)
Q5/50 ❌ Pred: C, Gold: (C)
Q6/50 ⚠️ Pred: None, Gold: (B)
Q7/50 ❌ Pred: D, Gold: (D)
Q8/50 ⚠️ Pred: None, Gold: (C)
Q9/50 ❌ Pred: C, Gold: (C)
Q10/50 ❌ Pred: B, Gold: (B)
  └─ 진행률: 10/50 (20.0%)
----------------------------------------
Q11/50 ❌ Pred: B, Gold: (B)
Q12/50 ❌ Pred: B, Gold: (B)
Q13/50 ❌ Pred: C, Gold: (C)
Q14/50 ❌ Pred: A, Gold: (C)
Q15/50 ❌ Pred: A, Gold: (A)
Q16/50 ❌ Pred: B, Gold: (B)
Q17/50 ❌ Pred: C, Gold: (B)
Q18/50 ❌ Pred: D, Gold: (D)
Q19/50 ❌ Pred: D, Gold: (D)
Q20/50 ❌ Pred: B, Gold: (D)
  └─ 진행률: 20/50 (40.0%)
----------------------------------------
Q21/50 ❌ Pred: B, Gold: (B)
Q22/50 ❌ Pred: A, Gold: (A)
Q23/50 ❌ Pred: B, Gold: (D)
Q24/50 ❌ Pred: C, Gold: (D)
Q25/50 ⚠️ Pred: None, Gold: (B)
Q26/50 ⚠️ Pred: None, Gold: (D)
Q27/50 ⚠️ Pred: None, Gold: (A)
Q28/50 ⚠️ Pred: None, Gold: (E)
Q29/50 ⚠️ 